In [2]:
from flask import Flask, render_template, request, json, jsonify

app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False

@app.route("/index", methods=['GET', 'POST'])
def index():
    if request.method == 'GET':
        return render_template('index.html')
    if request.method == 'POST':
        id = request.form.get('id')
        data, relat_id, us_na = find_relationship(id)
        #sentiment = get_sentiment(relat_id, us_na)
        return render_template('test.html', data=data)

#config
API_KEY = 'M16MB2eCRRvWCcfKDvr0qkxx5'
API_SECRET = 'UEt06p8FCjWOdOa784wiXgp0k9XspBOuMMrL50KSvv9YYQoPeP'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAOtVZAEAAAAAiG%2BIch2KP0AYd6MvmzDeCrjOO9s%3DxtpORldSCmg3V9TXGTZjhevpJHmqS5MhNM4FDG23ZIllelYJbu'
ACCESS_TOKEN = '1477449401719455745-j18PMElaCaxr2ZgPDx3xVMGuaULrdn'
ACCESS_TOKEN_SECRET = '8JR9KBNpQAUkH5HxnvnZxHhPCZC97oaAXbf6ihQ0Fk88M'

import tweepy
import time
client = tweepy.Client(bearer_token = BEARER_TOKEN)

def find_relationship(id):
    ids = [id]
    #get user_id
    users = client.get_users(usernames=id, user_fields = ['profile_image_url'])
    user_main = {}
    for user in users.data:
        USER_ID = user.id
        user_main_name = user.name
        user_main_url = user.profile_image_url

    #get user tweet_id
    tweet_id = []
    t = client.get_users_tweets(id = USER_ID, max_results = 50, tweet_fields = ['conversation_id', 'in_reply_to_user_id'])
    for i in t.data:
        tweet_id.append(i.id)

    #get user followers_id
    followers_id = []  
    tmp = []
    for response in tweepy.Paginator(client.get_users_followers, USER_ID, max_results=1000, limit=2):
        for i in response:
            tmp.append(i)
    for i in range(0, len(tmp), 4):
        for j in range(len(tmp[i])):
            followers_id.append(tmp[i][j].id)

    #get user following_id
    following_id = []
    tmp = []
    for response in tweepy.Paginator(client.get_users_following, USER_ID, max_results=1000, limit=2):
        for i in response:
            tmp.append(i)
    for i in range(0, len(tmp), 4):
        for j in range(len(tmp[i])):
            following_id.append(tmp[i][j].id)

    #get tweets reply_user
    reply_user = []
    for k in range(len(tweet_id)):
        query = 'conversation_id:' + str(tweet_id[k])
        l = client.search_recent_tweets(query = query, max_results = 50, expansions = ['author_id'])
        try:
            for i in l.data:
                reply_user.append(i.author_id)
        except:
            print("", end = "")
    
    #get tweets liking_users
    liking_users = []
    for k in range(len(tweet_id)):
        try:
            l = client.get_liking_users(id = tweet_id[k])
        except:
            print("sleep")
            time.sleep(60)
        try:
            for i in l.data:
                liking_users.append(i.id)
        except:
            print("", end = "")
    
    #get tweets retweeters
    retweeters = []
    for k in range(len(tweet_id)):
        try:
            l = client.get_retweeters(id = tweet_id[k])
        except:
            print("sleep")
            time.sleep(60)
        try:
            for i in l.data:
                retweeters.append(i.id)
        except:
            print("", end = "")
    

    #count relationship
    total = {}
    for i in liking_users:
        if i in total.keys():
            total[i] += 2
        else:
            total[i] = 2

    for i in retweeters:
        if(i in total.keys()):
            total[i] += 2
        else:
            total[i] = 2

    for i in reply_user:
        if i in total.keys():
            total[i] += 3
        else:
            total[i] = 3
    
    total[USER_ID] = 0
    
    result = sorted(total.items(), key=lambda x:x[1])[-10:]

    result_rev = []
    for i in range(len(result)-1, 0, -1):
        result_rev.append(result[i])
    
    #top 10 user_data
    related_id = {}
    user_name = {}
    users_name_url = {}
    for i in result_rev:
        u = client.get_users(ids = i[0], user_fields = ['profile_image_url'])
        for k in u.data:
            related_id[k.id] = []
            user_name[k.id] = k.name
            users_name_url[str(k.id)] = {"name":k.name, "url":k.profile_image_url}
    users_name_url[str(USER_ID)] = {"name":user_main_name, "url":user_main_url}
    for i in users_name_url:
        users_name_url[i]['url'] = users_name_url[i]['url'].replace("normal.jpg", "400x400.jpg")
    for i in users_name_url:
        print(users_name_url[i]['name'])
    return users_name_url, related_id, user_name
    
"""
import re

from googletrans import Translator
translator = Translator()

#Removeing @...
def remove_pattern(text):
    return re.sub('@[\w]*', '', text)

#Removeing http
def remove_http(text):
    results = re.compile(r'[http|https]*://[a-zA-Z0-9._?/&=:]*', re.S) 
    return re.sub(results, '', text)

#Removing the noisy text
def denoise_text(text):
    text = remove_pattern(text)
    text = remove_http(text)
    return text


import numpy as np
import torch 
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

class Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

token = BertTokenizer.from_pretrained('bert-base-chinese')
model = torch.load('model3.pth',map_location ='cpu')

def collate_fn(data):
    sents = [data[i][0] for i in range(len(data))]
    labels = [data[i][1] for i in range(len(data))]
    
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                    truncation=True, 
                    padding='max_length',
                    max_length=50,
                    return_tensors='pt',
                    return_length=True)
    
    input_ids = data['input_ids'] 
    attention_mask = data['attention_mask'] 
    token_type_ids = data['token_type_ids'] 
    labels = torch.LongTensor(labels) 
    
    return input_ids, attention_mask, token_type_ids, labels

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    tmp = []
    with torch.no_grad():
        for data in dataloader:
            
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            tmp.append(pred[0])
    return tmp

def get_sentiment(related_id, user_name):
    for j in related_id.keys():
        t = client.get_users_tweets(id = j, max_results = 50, tweet_fields = ["referenced_tweets"])
        try:
            for i in t.data:
                if i.referenced_tweets != None:
                    if i.referenced_tweets[0].type == "retweeted" :
                        a = client.get_tweet(id = i.referenced_tweets[0].id)
                        related_id[j].append(a.data.text)
                    else:
                        related_id[j].append(i.text)
                else:
                    related_id[j].append(i.text)
        except:
            print(end = "")
    for i in related_id:
        for j in range(len(related_id[i])):
            related_id[i][j] = denoise_text(related_id[i][j])
            text = related_id[i][j]
            try:
                related_id[i][j] = translator.translate(text, dest='zh-tw').text
            except:
                print(end = "")

    Sentiment_result = {}
    for i in related_id:
        test_text = np.array(related_id[i], dtype=object)
        test_target = np.array([-1 for b in range(len(related_id[i]))], dtype=np.int64)
        test_dataset = Dataset(test_text, test_target)
        testloader = DataLoader(dataset=test_dataset,batch_size=1,collate_fn=collate_fn,shuffle=False,drop_last=False)
        Sentiment_result[i] = get_predictions(model, testloader, compute_acc=True)
    for i in Sentiment_result:
        print("id:", user_name[i], "\n喜:", Sentiment_result[i].count(0), "怒:", Sentiment_result[i].count(1), "哀:", Sentiment_result[i].count(2),"中立:", Sentiment_result[i].count(3))
        print()
"""


'\nimport re\n\nfrom googletrans import Translator\ntranslator = Translator()\n\n#Removeing @...\ndef remove_pattern(text):\n    return re.sub(\'@[\\w]*\', \'\', text)\n\n#Removeing http\ndef remove_http(text):\n    results = re.compile(r\'[http|https]*://[a-zA-Z0-9._?/&=:]*\', re.S) \n    return re.sub(results, \'\', text)\n\n#Removing the noisy text\ndef denoise_text(text):\n    text = remove_pattern(text)\n    text = remove_http(text)\n    return text\n\n\nimport numpy as np\nimport torch \nfrom torch.utils.data import Dataset, DataLoader\nfrom transformers import BertTokenizer\n\nclass Dataset(Dataset):\n    def __init__(self, x, y):\n        self.x = x\n        self.y = y\n    \n    def __len__(self):\n        return len(self.x)\n    \n    def __getitem__(self, i):\n        return self.x[i], self.y[i]\n\ntoken = BertTokenizer.from_pretrained(\'bert-base-chinese\')\nmodel = torch.load(\'model3.pth\',map_location =\'cpu\')\n\ndef collate_fn(data):\n    sents = [data[i][0] for i in r

In [3]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jul/2022 17:44:39] "GET /index HTTP/1.1" 200 -
